In [1]:
## pip install pulp

from pulp import *
import pulp as plp
import pandas as pd

import pymysql
import pandas as pd

conn_alunos = pymysql.connect(
host = 'exe.cw39dq7h47z2.us-east-2.rds.amazonaws.com',
port = 3306,
user = 'alunos_exemplaria',
passwd = 'exemplariaaimf4',
database='wine'
)

In [2]:
## Query otimizador
query = """ SELECT

	ficha_tecnica.sku,
    
	(ficha_tecnica.temperatura * 1) as temperatura,
    
	(ficha_tecnica.potencial_de_guarda * 1) as potencial_de_guarda,
    
	(price.price_non_partner * 1) AS price_non_partner,
    
	review_rating.ratingValue AS review_rating
    
FROM ficha_tecnica 

LEFT JOIN price ON 
    price.sku = ficha_tecnica.sku
LEFT JOIN harmonizacao ON
	harmonizacao.sku = ficha_tecnica.sku
LEFT JOIN sommelier ON 
	sommelier.sku = ficha_tecnica.sku
LEFT JOIN review_rating ON 
	review_rating.sku = ficha_tecnica.sku
    
where review_rating.reviewCount >= 5 """
    
df = pd.read_sql(query,con=conn_alunos)
df = df.astype(float)

In [5]:
#definir o problema no pulp
prob = LpProblem("Maximizar o potencial de guarda respeitando a restricao que o review tem que ser maior que 4 e a temperatura tem que ser maior que 10 e o total de garrafas tem que ser igual a 100000",LpMaximize)

vinhos = df['sku'].tolist()
review = dict(zip(df['sku'],df['review_rating']))
potencial_de_guarda = dict(zip(df['sku'],df['potencial_de_guarda']))
temperatura = dict(zip(df['sku'],df['temperatura']))
preco = dict(zip(df['sku'],df['price_non_partner']))

/home/ronaldo/.local/share/virtualenvs/wino-rEbptTiD/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
#Garatir que os alimentos tem sempre porção positiva em geral os vinhos são produzidos numa safra de 5000, vamos supor que todos esse estão disponíveis para compra
vinhos_var = LpVariable.dicts("Vinhos",vinhos,lowBound=0,upBound=5000,cat='Integer')

#objetiva
prob += lpSum([potencial_de_guarda[i]*vinhos_var[i] for i in vinhos]) #note o sinal de menos para quando ele achar o minimo de -obj eh o maximo de obj

lpSum([potencial_de_guarda[i]*vinhos_var[i] for i in vinhos])
#constraints 1, review medio maior que 4
prob += lpSum([review[f] * vinhos_var[f] for f in vinhos]) >= 4*100000 # nota vezes qty vinhos

#constraints 2, temperatura maior que 10
prob += lpSum([temperatura[f] * vinhos_var[f] for f in vinhos]) >= 10*100000 # tempretura vezes qty vinhos

#constraints 3, capacidade total igual a 100000 
prob += lpSum([vinhos_var[f] for f in vinhos]) <= 100000
prob += lpSum([vinhos_var[f] for f in vinhos]) >= 100000

print(prob)

Maximizar_o_potencial_de_guarda_respeitando_a_restricao_que_o_review_tem_que_ser_maior_que_4_e_a_temperatura_tem_que_ser_maior_que_10_e_o_total_de_garrafas_tem_que_ser_igual_a_100000:
MAXIMIZE
4.0*Vinhos_10839.0 + 3.0*Vinhos_11610.0 + 6.0*Vinhos_11982.0 + 8.0*Vinhos_11983.0 + 3.0*Vinhos_12636.0 + 3.0*Vinhos_12733.0 + 3.0*Vinhos_12734.0 + 3.0*Vinhos_12786.0 + 5.0*Vinhos_12994.0 + 3.0*Vinhos_13296.0 + 4.0*Vinhos_13297.0 + 5.0*Vinhos_13639.0 + 5.0*Vinhos_14145.0 + 4.0*Vinhos_14146.0 + 5.0*Vinhos_14363.0 + 5.0*Vinhos_14755.0 + 6.0*Vinhos_15021.0 + 6.0*Vinhos_15026.0 + 5.0*Vinhos_15033.0 + 3.0*Vinhos_15034.0 + 6.0*Vinhos_15539.0 + 7.0*Vinhos_15806.0 + 5.0*Vinhos_16110.0 + 5.0*Vinhos_16111.0 + 6.0*Vinhos_16240.0 + 4.0*Vinhos_16363.0 + 6.0*Vinhos_16546.0 + 6.0*Vinhos_16547.0 + 6.0*Vinhos_16770.0 + 5.0*Vinhos_16862.0 + 6.0*Vinhos_16878.0 + 7.0*Vinhos_16879.0 + 5.0*Vinhos_16914.0 + 5.0*Vinhos_16959.0 + 3.0*Vinhos_17043.0 + 3.0*Vinhos_17061.0 + 8.0*Vinhos_17131.0 + 3.0*Vinhos_17269.0 + 8.0*Vinho

In [7]:
prob.solve()

1

In [8]:
list1 = []
list2 = []
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue > 0:
        print(v)
        print(v.varValue)
        list1.append(v)
        list2.append(v.varValue)
vinhos_selecionados = dict(zip(list1,list2))
print(vinhos_selecionados)

Status: Optimal
Vinhos_17131.0
5000.0
Vinhos_17631.0
1250.0
Vinhos_17635.0
5000.0
Vinhos_18411.0
5000.0
Vinhos_18615.0
5000.0
Vinhos_18941.0
5000.0
Vinhos_20117.0
5000.0
Vinhos_20150.0
5000.0
Vinhos_20181.0
5000.0
Vinhos_20295.0
5000.0
Vinhos_20297.0
3750.0
Vinhos_20606.0
5000.0
Vinhos_20607.0
5000.0
Vinhos_21552.0
5000.0
Vinhos_21768.0
5000.0
Vinhos_22046.0
5000.0
Vinhos_22102.0
5000.0
Vinhos_22411.0
5000.0
Vinhos_22476.0
5000.0
Vinhos_22658.0
5000.0
Vinhos_4592.0
5000.0
{Vinhos_17131.0: 5000.0, Vinhos_17631.0: 1250.0, Vinhos_17635.0: 5000.0, Vinhos_18411.0: 5000.0, Vinhos_18615.0: 5000.0, Vinhos_18941.0: 5000.0, Vinhos_20117.0: 5000.0, Vinhos_20150.0: 5000.0, Vinhos_20181.0: 5000.0, Vinhos_20295.0: 5000.0, Vinhos_20297.0: 3750.0, Vinhos_20606.0: 5000.0, Vinhos_20607.0: 5000.0, Vinhos_21552.0: 5000.0, Vinhos_21768.0: 5000.0, Vinhos_22046.0: 5000.0, Vinhos_22102.0: 5000.0, Vinhos_22411.0: 5000.0, Vinhos_22476.0: 5000.0, Vinhos_22658.0: 5000.0, Vinhos_4592.0: 5000.0}


In [10]:
obj = value(prob.objective)
print("O potencial de guarda da solução ótima é: {}".format(round(obj/100000,2)))
print("O potencial de guarda de guarda médio é: {}".format(round(df['potencial_de_guarda'].mean(),2)))


O potencial de guarda da solução ótima é: 9.68
O potencial de guarda de guarda médio é: 4.99
